# Custom Models, Layers, and Loss Functions with TensorFlow 
### *Week: 4*

#### DeepLearningAI

## Description

* Compare Functional and Sequential APIs, discover new models you can build with the Functional API, and build a model that produces multiple outputs including a Siamese network.
* Build custom loss functions (including the contrastive loss function used in a Siamese network) in order to measure how well a model is doing and help your neural network learn from training data. 
* Build off of existing standard layers to create custom layers for your models, customize a network layer with a lambda layer, understand the differences between them, learn what makes up a custom layer, and explore activation functions. 
* Build off of existing models to add custom functionality, learn how to define your own custom class instead of using the Functional or Sequential APIs, build models that can be inherited from the TensorFlow Model class, and build a residual network (ResNet) through defining a custom model class. 


The DeepLearning.AI TensorFlow: Advanced Techniques Specialization introduces the features of TensorFlow that provide learners with more control over their model architecture and tools that help them create and train advanced ML models.  

This Specialization is for early and mid-career software and machine learning engineers with a foundational understanding of TensorFlow who are looking to expand their knowledge and skill set by learning advanced TensorFlow features to build powerful models.

## Complex Architectures with the Function API

![Designed-network](newtork.png)

```Python
input_a = Input(shape=[1], name="Wide_Input")
input_b = Input(shape=[1], name="Deep_Input")
hidden_1 = Dense(30, activation="relu")(input_b)
hidden_2 = Dense(30, activation="relu")(hidden_1)
concat = concatenate([input_a, hidden_2])
output = Dense(1, name="Output",)(concat) # You can also add second out layer.

model = Model(inputs=[input_a, input_b], outputs=[output]) # If you added second out layer, outputs list will be also changed.
```

![other-model](other_model.png)
**This image different with above code, (second output layer)**

```Python
class WideAndDeepModel(Model):
    def __init__(self, units=30, activation='relu', **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = Dense(units, activation=activation)
        self.hidden2 = Dense(units, activation=activation)
        self.main_output = Dense(1)
        self.aux_output = Dense(1)
        
    def call(self, inputs):
        input_A, input_B = inputs
        hidden1 = self.hidden1(input_B)
        hidden2 = self.hidden2(hidden1)
        concat = concatenate([input_A, hidden2])
        main_output = self.main_output(concat)
        aux_output = self.aux_output(hidden2)
        return main_output, aux_output

# The basic usage is

model = WideAndDeepModel()
```


### [The first lab notebook](C1_W4_Lab_1_basic-model.ipynb)

## Using the Model Class to simplify Complex Architecture

#### The Model Class

* Built-in training, evaluation, and prediction loops
    
`model.fit()`, `model.evaluate()`, `model.predict()`
    
* Saving and serialization APIs

`model.save()`, `model.save_weights()`
    
* Summarization and visualization APIs

`model.summary()`, `tf.keras.utils.plot_model`

#### Limitations of Sequential/Functional APIs

* Only suited to models that are Directed Acyclic Graphs of layers

>In these cases, the data flows from the inputs to the outputs, and sometimes it's in multiple branches as we've seen already, but the direction is always the same, it never loops back during training or inference. 

`MobilNet`, `Inception`, `etc.`

* More exotic architecture

>Thus networks where recursion is used or dynamic networks where the architecture can change on the fly can be very difficult to build if you use these APIs. With sequential, they're impossible, with a functional APIs, they might be possible, but will involve a lot of coding. But with modal subclassing, it can become a bit easier to achieve these complex scenarios.

`dynamic and recursive networks`

#### Benefits of subclassing models

* Extends how you've been building models

* Continue to use functional and sequential code

* Modular Architecture

* Try out experiments quickly

* Control flow in the network

### Understanding Residual Networks

![resnet_1](resnet1.png)

![resnet_2](resnet2.png)

![resnet_3](resnet3.png)

```Python
class CNNResidual(Layer):
    def __init__(self, layers, filters, **kwargs):
        super()__init__(**kwargs)
        self.hidden = [Conv2D(filters, (3, 3), activation='relu') for _ in range(layers)]
        
    def call(self, inputs):
        x = inputs
        for layer in self.hidden:
            x = layer(x)
        return inputs + x
```

```Python

class DNNResidual(Layer):
    def __init__(selfi layers, neurons, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [Dense(neurons, activation="relu") for _ in range(layers)]

    def call(self, inputs):
        x = inputs
        for layer in self.hidden:
            x = layer(x)
        return inputs + x
```

![resnet_4](resnet4.png)


```Python
class MyResidual(Model):
    def __init__(self, **kwargs):
        self.hidden1 = Dense(30, activation='relu')
        self.block1 = CNNResidual(2, 32)
        self.block2 = DNNResidual(2, 64)
        self.out = Dense(1)
        
        
    def call(self, inputs):
        x = self.hidden1(inputs)
        x = self.block1(x)
        for _ in range(1, 4):
            x = self.block2(x)
            
        return self.out(x)
```


## Implementing ResNet 

### Coding a Residual network with the Model class


**ResNet**

![Big-ResNet](resnet4-1.png)

![Identity-Block](resnet4-2.png)

![Identity ResNet Block with 1x1 Conv](resnet4-3.png)

![Mini-ResNet](resnet4-4.png)


#### We will be coding mini-ResNet model

```Python
class IdentityBlock(tf.keras.Model):
    def __init__(self, filters, kernel_size):
        super(IdentityBlock, self).__init__(name='')
        
        self.conv1 = tf.keras.layers.Conv2D(filters, kernel_size, padding='same')
        self.bn1 = tf.keras.layers.BatchNormalization()
        
        self.conv2 = tf.keras.layers.Conv2D(filters, kernel_size, padding='same')
        self.bn2 = tf.keras.layers.BatchNormalization()
        
        self.act = tf.keras.layers.Activation('relu')
        self.add = tf.keras.layers.Add()
        
        
    def call(self, input_tensor):
        x = self.conv1(input_tensor)
        x = self.bn1(x)
        x = self.act(x)
        
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.act(x)
        
        x = self.add([x, input_tensor])
        x = self.act(x)
        
        return x
```


```Python
class MiniResNet(tf.keras.Model):
    def __init__(self, num_classes):
        super(MiniResNet, self).__init__()
        self.conv = tf.keras.layers.Conv2D(64, 7, padding='same')
        self.bn = tf.keras.layers.BatchNormalization()
        self.act = tf.keras.layers.Activation('relu')
        self.max_pool = tf.keras.layers.MaxPool2D((2, 2))
        self.id1a = IdentityBlock(64, 3)
        self.id1b = IdentityBlock(64, 3)
        self.global_pool = tf.keras.layers.GlobalAveragePooling2D()
        self.classifier = tf.keras.layers.Dense(num_classes, activation='softmax')
        
        
    def call(self, inputs):
        x = self.conv(inputs)
        x = self.bn(x)
        x = self.act(x)
        x = self.max_pool(x)
        
        x = self.id1a(x)
        x = self.id1b(x)
        
        x = self.global_pool(x)
        return self.classifier(x)
    
# USAGE OF THE MODEL

resnet = MiniResNet(10)
resnet.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
dataset = tfds.load('mnist', split=tfds.Split.TRAIN)
dataset = dataset.map(preprocess).batch(32)

resnet.fit(dataset, epochs=1)
```

### [The second lab notebook](C1_W4_Lab_2_resnet-example.ipynb)